In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchaudio
import torchaudio.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [2]:
!wget https://github.com/karoldvl/ESC-50/archive/master.zip

--2025-08-23 11:17:30--  https://github.com/karoldvl/ESC-50/archive/master.zip
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/karolpiczak/ESC-50/archive/master.zip [following]
--2025-08-23 11:17:30--  https://github.com/karolpiczak/ESC-50/archive/master.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master [following]
--2025-08-23 11:17:30--  https://codeload.github.com/karolpiczak/ESC-50/zip/refs/heads/master
Resolving codeload.github.com (codeload.github.com)... 140.82.116.10
Connecting to codeload.github.com (codeload.github.com)|140.82.116.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘master.zip’

master.zip              [  <

In [3]:
!unzip master.zip

Archive:  master.zip
33c8ce9eb2cf0b1c2f8bcf322eb349b6be34dbb6
   creating: ESC-50-master/
   creating: ESC-50-master/.circleci/
  inflating: ESC-50-master/.circleci/config.yml  
   creating: ESC-50-master/.github/
  inflating: ESC-50-master/.github/stale.yml  
 extracting: ESC-50-master/.gitignore  
  inflating: ESC-50-master/LICENSE   
  inflating: ESC-50-master/README.md  
   creating: ESC-50-master/audio/
  inflating: ESC-50-master/audio/1-100032-A-0.wav  
  inflating: ESC-50-master/audio/1-100038-A-14.wav  
  inflating: ESC-50-master/audio/1-100210-A-36.wav  
  inflating: ESC-50-master/audio/1-100210-B-36.wav  
  inflating: ESC-50-master/audio/1-101296-A-19.wav  
  inflating: ESC-50-master/audio/1-101296-B-19.wav  
  inflating: ESC-50-master/audio/1-101336-A-30.wav  
  inflating: ESC-50-master/audio/1-101404-A-34.wav  
  inflating: ESC-50-master/audio/1-103298-A-9.wav  
  inflating: ESC-50-master/audio/1-103995-A-30.wav  
  inflating: ESC-50-master/audio/1-103999-A-30.wav  
  infla

In [4]:
class ESC50Dataset(Dataset):
    def __init__(self, data_dir, metadata_file, split="train", transform=None):
        super().__init__()
        self.data_dir = Path(data_dir)
        self.metadata = pd.read_csv(metadata_file)
        self.split = split
        self.transform = transform
        if split == "train":
            self.metadata = self.metadata[self.metadata["fold"] != 5]
        else:
            self.metadata = self.metadata[self.metadata["fold"] == 5]
        self.classes = sorted(self.metadata["category"].unique())
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.metadata["label"] = self.metadata["category"].map(self.class_to_idx)

    def __len__(self):
        return len(self.metadata)

    def __getitem__(self, idx):
        row = self.metadata.iloc[idx]
        audio_path = self.data_dir / "audio" / row["filename"]
        waveform, sample_rate = torchaudio.load(audio_path)
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        if self.transform:
            spectrogram = self.transform(waveform)
        else:
            spectrogram = waveform
        return spectrogram, row["label"]

In [5]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.shortcut = nn.Sequential()
        self.use_shortcut = stride != 1 or in_channels != out_channels
        if self.use_shortcut:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

    def forward(self, x):
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        shortcut = self.shortcut(x) if self.use_shortcut else x
        out_add = out + shortcut
        return out_add


class Au57(nn.Module):
    def __init__(self, num_classes=50):
        super(Au57, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 64, 7, 2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
        self.res_layer1 = nn.ModuleList([ResidualBlock(64, 64) for _ in range(3)])
        self.res_layer2 = nn.ModuleList(
            [ResidualBlock(64 if i == 0 else 128, 128, stride=2 if i == 0 else 1) for i in range(4)]
        )
        self.res_layer3 = nn.ModuleList(
            [
                ResidualBlock(128 if i == 0 else 256, 256, stride=2 if i == 0 else 1)
                for i in range(6)
            ]
        )
        self.res_layer4 = nn.ModuleList(
            [
                ResidualBlock(256 if i == 0 else 512, 512, stride=2 if i == 0 else 1)
                for i in range(3)
            ]
        )
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        for block in self.res_layer1:
            x = block(x)
        for block in self.res_layer2:
            x = block(x)
        for block in self.res_layer3:
            x = block(x)
        for block in self.res_layer4:
            x = block(x)
        x = self.avg_pool(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [6]:
def mixup_data(x, y):
    lam = np.random.beta(0.2, 0.2)
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [7]:
train_transform = nn.Sequential(
    transforms.MelSpectrogram(
        sample_rate=22050,
        n_fft=1024,
        hop_length=512,
        n_mels=128,
        f_min=0,
        f_max=11025,
    ),
    transforms.AmplitudeToDB(),
    transforms.FrequencyMasking(freq_mask_param=30),
    transforms.TimeMasking(time_mask_param=80),
)

val_transform = nn.Sequential(
    transforms.MelSpectrogram(
        sample_rate=22050,
        n_fft=1024,
        hop_length=512,
        n_mels=128,
        f_min=0,
        f_max=11025,
    ),
    transforms.AmplitudeToDB(),
)

In [8]:
esc50_dir = Path("/content/ESC-50-master")  # adjust if you upload ESC-50 to Colab
train_dataset = ESC50Dataset(
    data_dir=esc50_dir,
    metadata_file=esc50_dir / "meta" / "esc50.csv",
    split="train",
    transform=train_transform,
)
val_dataset = ESC50Dataset(
    data_dir=esc50_dir,
    metadata_file=esc50_dir / "meta" / "esc50.csv",
    split="test",
    transform=val_transform,
)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [9]:
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of validation samples: {len(val_dataset)}")

Number of training samples: 1600
Number of validation samples: 400


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Au57(num_classes=len(train_dataset.classes)).to(device)
epochs = 100
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=0.002,
    epochs=epochs,
    steps_per_epoch=len(train_dataloader),
    pct_start=0.1,
)

best_acc = 0.0
writer = SummaryWriter("/content/tensorboard_logs")

for epoch in range(epochs):
    model.train()
    epoch_loss = 0.0
    progress_bar = tqdm(train_dataloader, desc=f"Epoch {epoch + 1}/{epochs}")
    for data, target in progress_bar:
        data, target = data.to(device), target.to(device)
        if np.random.random() > 0.7:
            data, target_a, target_b, lam = mixup_data(data, target)
            output = model(data)
            loss = mixup_criterion(criterion, output, target_a, target_b, lam)
        else:
            output = model(data)
            loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        epoch_loss += loss.item()
        progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

    avg_epoch_loss = epoch_loss / len(train_dataloader)
    writer.add_scalar("Loss/Train", avg_epoch_loss, epoch)

    model.eval()
    correct, total = 0, 0
    val_loss = 0
    with torch.no_grad():
        for data, target in val_dataloader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            val_loss += criterion(output, target).item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    acc = 100 * correct / total
    avg_val_loss = val_loss / len(val_dataloader)
    writer.add_scalar("Loss/Validation", avg_val_loss, epoch)
    writer.add_scalar("Accuracy/Validation", acc, epoch)
    print(
        f"Epoch {epoch + 1} Loss: {avg_epoch_loss:.4f}, "
        f"Val Loss: {avg_val_loss:.4f}, Accuracy: {acc:.2f}%"
    )

    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), "/content/drive/MyDrive/models/best_model.pth")
        print(f"New best model saved: {acc:.2f}%")

Epoch 1/100:   0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/a

Epoch 1 Loss: 3.8134, Val Loss: 5.4578, Accuracy: 7.25%
New best model saved: 7.25%


Epoch 2/100: 100%|██████████| 50/50 [00:55<00:00,  1.10s/it, Loss=3.0055]


Epoch 2 Loss: 3.3114, Val Loss: 7.6948, Accuracy: 8.25%
New best model saved: 8.25%


Epoch 3/100: 100%|██████████| 50/50 [00:55<00:00,  1.11s/it, Loss=3.1862]


...

Epoch 98 Loss: 1.0911, Val Loss: 1.3787, Accuracy: 83.00%


Epoch 99/100: 100%|██████████| 50/50 [00:54<00:00,  1.09s/it, Loss=0.8061]


Epoch 99 Loss: 0.9013, Val Loss: 1.3822, Accuracy: 82.75%


Epoch 100/100: 100%|██████████| 50/50 [00:54<00:00,  1.09s/it, Loss=2.2072]


Epoch 100 Loss: 1.0475, Val Loss: 1.3976, Accuracy: 81.25%
